In [16]:
import time
import random
import requests

import signaturehelper

In [ ]:
# !pip install requests==2.21.0 

In [2]:
!pip show requests

Name: requests
Version: 2.21.0
Summary: Python HTTP for Humans.
Home-page: http://python-requests.org
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: c:\users\epsilon\dev\anaconda\envs\kera\lib\site-packages
Requires: idna, chardet, certifi, urllib3
Required-by: tweepy, tensorboard, requests-oauthlib, pandas-profiling, jupyterlab-server, folium


In [28]:
get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID)

{'Content-Type': 'application/json; charset=UTF-8',
 'X-Timestamp': '1612338453856',
 'X-API-KEY': '01000000002c6573aff319a7fb49a48153878c0e3559a4cddf9da1a4e404cb94b9716d9b85',
 'X-Customer': '1930249',
 'X-Signature': b'RyUG4nE9gvuWgv0HR1vbX+DIBoo/MZt9G2y50vxt7rM='}

In [46]:
# it has bug now. they don't match with real keys
keydict = {}
with open('./keys.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        keyname = line.split('=')[0]
        key = line.split('=',maxsplit=1)[1].strip()
        keydict[keyname] = key
    

In [24]:
API_KEY = keydict['API_KEY']
SECRET_KEY = keydict['SECRET_KEY']
CUSTOMER_ID = int(keydict['CUSTOMER_ID'])

In [40]:
line

'CUSTOMER_ID=1930249'

In [48]:
keydict['SECRET_KEY'] == SECRET_KEY

True

In [51]:
keydict['CUSTOMER_ID'] = 

'1930249'

In [38]:
SECRET_KEY

'AQAAAAAsZXOv8xmn+0mkgVOHjA41adczLrAq2j84OKjr0N/NNA=='

In [50]:
print(API_KEY==keydict['API_KEY'] )
print(SECRET_KEY==keydict['SECRET_KEY'])
print(CUSTOMER_ID==keydict['CUSTOMER_ID'])

True
True
False


In [30]:
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {
            'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID), 
            'X-Signature': signature
           }

BASE_URL = 'https://api.naver.com'


API_KEY='01000000002c6573aff319a7fb49a48153878c0e3559a4cddf9da1a4e404cb94b9716d9b85'
SECRET_KEY='AQAAAAAsZXOv8xmn+0mkgVOHjA41adczLrAq2j84OKjr0N/NNA=='
CUSTOMER_ID=1930249
uri = '/keywordstool'
method = 'GET'
payload={
        'hintKeywords':'쌍화차',
        'showDetail' :1
        }
# {'type': 'MYCLIENTS'}
r = requests.get(BASE_URL + uri, params=payload, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
# json = payload #if you use 
print("response status_code = {}".format(r.status_code) )
print("response body = {}".format(r.json() )  )

response status_code = 200
response body = {'keywordList': [{'relKeyword': '쌍화차', 'monthlyPcQcCnt': 7120, 'monthlyMobileQcCnt': 56900, 'monthlyAvePcClkCnt': 51.7, 'monthlyAveMobileClkCnt': 1589.8, 'monthlyAvePcCtr': 0.78, 'monthlyAveMobileCtr': 2.85, 'plAvgDepth': 15, 'compIdx': '높음'}, {'relKeyword': '생강차', 'monthlyPcQcCnt': 4770, 'monthlyMobileQcCnt': 25100, 'monthlyAvePcClkCnt': 16.7, 'monthlyAveMobileClkCnt': 432.6, 'monthlyAvePcCtr': 0.37, 'monthlyAveMobileCtr': 1.76, 'plAvgDepth': 15, 'compIdx': '높음'}, {'relKeyword': '한방차', 'monthlyPcQcCnt': 1980, 'monthlyMobileQcCnt': 2990, 'monthlyAvePcClkCnt': 11.3, 'monthlyAveMobileClkCnt': 58.8, 'monthlyAvePcCtr': 0.6, 'monthlyAveMobileCtr': 2.1, 'plAvgDepth': 15, 'compIdx': '높음'}, {'relKeyword': '대추차', 'monthlyPcQcCnt': 2340, 'monthlyMobileQcCnt': 14300, 'monthlyAvePcClkCnt': 9.1, 'monthlyAveMobileClkCnt': 241.9, 'monthlyAvePcCtr': 0.42, 'monthlyAveMobileCtr': 1.76, 'plAvgDepth': 15, 'compIdx': '높음'}, {'relKeyword': '쌍화차재료', 'monthlyPcQcCnt'

In [17]:
reqdt = r.json()

In [22]:
len(reqdt['keywordList'])

885

In [21]:
reqdt['keywordList']

[{'relKeyword': '쌍화차',
  'monthlyPcQcCnt': 7120,
  'monthlyMobileQcCnt': 56900,
  'monthlyAvePcClkCnt': 51.7,
  'monthlyAveMobileClkCnt': 1589.8,
  'monthlyAvePcCtr': 0.78,
  'monthlyAveMobileCtr': 2.85,
  'plAvgDepth': 15,
  'compIdx': '높음'},
 {'relKeyword': '생강차',
  'monthlyPcQcCnt': 4770,
  'monthlyMobileQcCnt': 25100,
  'monthlyAvePcClkCnt': 16.7,
  'monthlyAveMobileClkCnt': 432.6,
  'monthlyAvePcCtr': 0.37,
  'monthlyAveMobileCtr': 1.76,
  'plAvgDepth': 15,
  'compIdx': '높음'},
 {'relKeyword': '한방차',
  'monthlyPcQcCnt': 1980,
  'monthlyMobileQcCnt': 2990,
  'monthlyAvePcClkCnt': 11.3,
  'monthlyAveMobileClkCnt': 58.8,
  'monthlyAvePcCtr': 0.6,
  'monthlyAveMobileCtr': 2.1,
  'plAvgDepth': 15,
  'compIdx': '높음'},
 {'relKeyword': '대추차',
  'monthlyPcQcCnt': 2340,
  'monthlyMobileQcCnt': 14300,
  'monthlyAvePcClkCnt': 9.1,
  'monthlyAveMobileClkCnt': 241.9,
  'monthlyAvePcCtr': 0.42,
  'monthlyAveMobileCtr': 1.76,
  'plAvgDepth': 15,
  'compIdx': '높음'},
 {'relKeyword': '쌍화차재료',
  'mont

#### response json Structure
{
    "relKeyword": "쌍화탕",
    "monthlyPcQcCnt": ">10",
    "monthlyMobileQcCnt": ">10",
    "monthlyAvePcClkCnt": ">10",
    "monthlyAveMobileClkCnt": ">10",
    "monthlyAvePcCtr": ">10",
    "monthlyAveMobileCtr": ">10",
    "plAvgDepth": ">10",
    "compIdx": "high"
}

In [9]:
r.text

'{"keywordList":[]}'

In [8]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [ ]:
# BusinessChannel Usage Sample

uri = '/ncc/channels'
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# Adgroup Usage Sample

# 1. GET adgroup Usage Sample

uri = '/ncc/adgroups'
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))
target_adgroup = r.json()[0]

# 2. CREATE adgroup Usage Sample

uri = '/ncc/adgroups'
method = 'POST'
payload = {'name': 'TEST#' + str(random.randrange(1000, 9999)), 'nccCampaignId' : target_adgroup['nccCampaignId'], 'pcChannelId' : target_adgroup['pcChannelId'], 'mobileChannelId': target_adgroup['mobileChannelId']}
r = requests.post(BASE_URL + uri, json=payload, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

created_adgroup = r.json()

# 3. UPDATE Adgroup Usage Sample

uri = '/ncc/adgroups/' + created_adgroup['nccAdgroupId']
method = 'PUT'
created_adgroup['userLock'] = 0
r = requests.put(BASE_URL + uri, params={'fields': 'userLock'}, json=created_adgroup, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 4. DELETE Adgroup

uri = '/ncc/adgroups/' + created_adgroup['nccAdgroupId']
method = 'DELETE'
r = requests.delete(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.content))

# AdKeyword Usage Sample

# 1. CREATE AdKeyword

uri = '/ncc/keywords'
method = 'POST'
r = requests.post(BASE_URL + uri, params={'nccAdgroupId': created_adgroup['nccAdgroupId']}, json=[{'keyword': 'hello'}], headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

created_adkeyword = r.json()[0]

# 2. GET AdKeyword

uri = '/ncc/keywords'
method = 'GET'
r = requests.get(BASE_URL + uri, params={'nccAdgroupId': created_adgroup['nccAdgroupId']}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 3. UPDATE AdKeyword

uri = '/ncc/keywords'
method = 'PUT'
created_adkeyword['userLock'] = 0
r = requests.put(BASE_URL + uri, params={'fields': 'userLock'}, json=[created_adkeyword], headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# 4. GET and UPDATE AdKeyword (BidAmt)

uri = '/ncc/keywords/' + created_adkeyword['nccKeywordId']
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

retrieved_adkeyword = r.json()

uri = '/ncc/keywords'
method = 'PUT'
retrieved_adkeyword['bidAmt'] = 300
retrieved_adkeyword['useGroupBidAmt'] = 0
r = requests.put(BASE_URL + uri, params={'fields': 'bidAmt'}, json=[retrieved_adkeyword], headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# 5. DELETE AdKeyword

uri = '/ncc/keywords/' + created_adkeyword['nccKeywordId']
method = 'DELETE'
r = requests.delete(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.content))


# Estimate Usage Sample

# 1. average-position-bid

uri = '/estimate/average-position-bid/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'items': [{'key': '제주여행', 'position': 1}, {'key': '게스트하우스', 'position': 2}, {'key': '자전거여행', 'position': 3}]}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("#response status_code = {}".format(r.status_code))
print("#response body = {}".format(r.json()))


# 2. exposure-minimum-bid

uri = '/estimate/exposure-minimum-bid/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'period': 'MONTH', 'items': ['제주여행', '게시트하우스', '자전거여행']}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 3. median-bid

uri = '/estimate/median-bid/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'period': 'MONTH', 'items': ['제주여행', '게시트하우스', '자전거여행']}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# 4. performance

uri = '/estimate/performance/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'keywordplus': True, 'key': '중고차', 'bids': [100, 500, 1000, 1500, 2000, 3000, 5000]}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 5. performance-bulk

uri = '/estimate/performance-bulk'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'items': [{'device': 'PC', 'keywordplus': True, 'keyword': '제주여행', 'bid': 70}, {'device': 'PC', 'keywordplus': True, 'keyword': '제주도', 'bid': 80}, {'device': 'PC', 'keywordplus': True, 'keyword': '제주도맛집', 'bid': 90}, ]}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# Stat Usage Sample

# 1. GET Summary Report per multiple entities 

uri = '/stats'
method = 'GET'
stat_ids = [target_adgroup['nccCampaignId'], target_adgroup['nccAdgroupId']]
r = requests.get(BASE_URL + uri, params={'ids': stat_ids, 'fields': '["clkCnt","impCnt","salesAmt", "ctr", "cpc", "avgRnk", "ccnt"]', 'timeRange': '{"since":"2019-06-01","until":"2019-06-25"}'}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))